In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm, skew, kurtosis
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# --- CONFIGURATION ---
FILE_RETURNS = "MASTER_RETURNS.csv"
CONFIDENCE_LEVEL = 0.95 
RISK_FREE_RATE = 0.0     
TRADING_DAYS = 252       
MC_SIMULATIONS = 10000   
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 10)


In [3]:
def load_data(filepath):
    return pd.read_csv(filepath, index_col=0, parse_dates=True)

In [4]:
def calc_annual_volatility(df):
    """Calculates Annualized Volatility (Standard Deviation * sqrt(252))."""
    return df.std() * np.sqrt(TRADING_DAYS)

In [5]:
def calc_semi_deviation(df):
    """Calculates Downside Risk (Standard Deviation of negative returns only)."""
    downside_returns = df[df < 0]
    return downside_returns.std() * np.sqrt(TRADING_DAYS)

In [6]:
def calc_ewma_volatility(df, alpha=0.06):
    """Calculates Exponentially Weighted Moving Average Volatility."""
    # alpha 0.06 corresponds roughly to lambda 0.94
    return df.ewm(alpha=alpha).std().iloc[-1] * np.sqrt(TRADING_DAYS)

In [7]:
def calc_ratio_symetrie(vol_ann, semi_dev):
    return vol_ann / semi_dev

In [8]:
def calc_var_historical(df, confidence=CONFIDENCE_LEVEL):
    """Calculates Value at Risk using the Historical method."""
    return df.quantile(1 - confidence)

In [9]:
def calc_var_parametric(df, confidence=CONFIDENCE_LEVEL):
    """Calculates Value at Risk using the Variance-Covariance (Gaussian) method."""
    mu = df.mean()
    sigma = df.std()
    z_score = norm.ppf(1 - confidence)
    return mu + z_score * sigma

In [10]:
def calc_var_monte_carlo(df, confidence=CONFIDENCE_LEVEL, simulations=MC_SIMULATIONS):
    """Calculates Value at Risk using Monte Carlo simulations."""
    mu = df.mean()
    sigma = df.std()
    var_mc = []
    for col in df.columns:
        sims = np.random.normal(mu[col], sigma[col], simulations)
        var_mc.append(np.percentile(sims, (1 - confidence) * 100))
    return pd.Series(var_mc, index=df.columns)

In [11]:
def calc_cvar(df, confidence=CONFIDENCE_LEVEL):
    """Calculates Conditional VaR (Expected Shortfall)."""
    var_hist = df.quantile(1 - confidence)
    cvar_list = []
    for col in df.columns:
        cutoff = var_hist[col]
        losses = df[col][df[col] <= cutoff]
        cvar_list.append(losses.mean())
    return pd.Series(cvar_list, index=df.columns)

In [12]:
def calc_annual_return(df):
    """Calculates Annualized Mean Return (Arithm)."""
    return df.mean() * TRADING_DAYS

In [13]:
def calc_sharpe_ratio(mean_ret, vol_ann, risk_free=RISK_FREE_RATE):
    """Calculates Sharpe Ratio."""
    return (mean_ret - risk_free) / vol_ann

In [14]:
def calc_sortino_ratio(mean_ret, semi_dev, risk_free=RISK_FREE_RATE):
    """Calculates Sortino Ratio."""
    return (mean_ret - risk_free) / semi_dev

In [15]:
def calc_beta(df):
    """Calculates Beta against an equal-weighted portfolio of all assets."""
    market_index = df.mean(axis=1)
    market_var = market_index.var()
    betas = []
    for col in df.columns:
        cov = df[col].cov(market_index)
        beta = cov / market_var if market_var != 0 else 0
        betas.append(beta)
    return pd.Series(betas, index=df.columns)

In [16]:
def calc_skewness(df):
    """Calculates Skewness (Asymmetry)."""
    return df.apply(skew)

In [17]:
def calc_kurtosis(df):
    """Calculates Excess Kurtosis (Tail thickness)."""
    return df.apply(kurtosis)

In [18]:
def calc_max_drawdown(df):
    """Calculates the Maximum Drawdown from peak."""
    nav = (1 + df).cumprod() * 100
    running_max = nav.cummax()
    drawdown = (nav - running_max) / running_max
    return drawdown.min()

In [19]:
def calc_calmar_ratio(mean_ret, max_dd):
    """Calculates Calmar Ratio (Annual Return / Abs Max Drawdown)."""
    return mean_ret / abs(max_dd)

In [20]:
def analyze_dataframe(df):
    """Computes all metrics for a given dataframe slice."""
    
    # Volatility
    vol_ann = calc_annual_volatility(df)
    semi_dev = calc_semi_deviation(df)
    ewma_vol = calc_ewma_volatility(df)
    ratio_sym = calc_ratio_symetrie(vol_ann, semi_dev)
    
    # Tail Risk
    var_hist = calc_var_historical(df)
    var_param = calc_var_parametric(df)
    var_mc = calc_var_monte_carlo(df)
    cvar = calc_cvar(df)
    
    # Performance
    mean_ret = calc_annual_return(df)
    sharpe = calc_sharpe_ratio(mean_ret, vol_ann)
    sortino = calc_sortino_ratio(mean_ret, semi_dev)
    beta = calc_beta(df)
    
    # Distribution
    skew_s = calc_skewness(df)
    kurt_s = calc_kurtosis(df)
    max_dd = calc_max_drawdown(df)
    calmar = calc_calmar_ratio(mean_ret, max_dd)

    # Assemble Report
    report = pd.DataFrame({
        'Volatilite (Ann)': vol_ann,
        'Semi-Deviation': semi_dev,
        'EWMA (Actuelle)': ewma_vol,
        'Ratio Symetrie (Vol/Semi)': ratio_sym,
        
        f'VaR Hist {CONFIDENCE_LEVEL:.0%}': var_hist,
        f'VaR Param {CONFIDENCE_LEVEL:.0%}': var_param,
        f'VaR MonteCarlo {CONFIDENCE_LEVEL:.0%}': var_mc,
        'CVaR (Expected Shortfall)': cvar,
        
        'Rendement Moyen (Ann)': mean_ret,
        'Ratio Sharpe': sharpe,
        'Ratio Sortino': sortino,
        'Beta (vs Market Avg)': beta,
        
        'Skewness': skew_s,
        'Kurtosis (Excess)': kurt_s,
        'Max Drawdown': max_dd,
        'Ratio Calmar': calmar
    })
    
    return report

In [21]:
def main():
    print(">>> STARTING MULTI-TIMEFRAME ANALYSIS (1Y, 5Y, FULL) <<<")
    
    # 1. Load Data
    df_full = load_data(FILE_RETURNS)
    # Define Periods
    # Note: TRADING_DAYS = 252
    periods = {
        '1Y': 252,
        '5Y': 252 * 5,
        'FULL': len(df_full) 
    }

    consolidated_report = pd.DataFrame()
    
    #Iterate over periods
    for name, length in periods.items():
        
        df_slice = df_full
        actual_days = len(df_full)
        # Run Analysis
        report = analyze_dataframe(df_slice)
        
        # Save Correlation Matrix for this period
        corr_filename = f"MATRICE_CORRELATION_{name}.csv"
        df_slice.corr().to_csv(corr_filename)
        print(f"   -> Saved {corr_filename}")
        
        # Rename columns to include period (e.g. "Ratio Sharpe [1Y]")
        report_renamed = report.copy()
        report_renamed.columns = [f"{col} [{name}]" for col in report.columns]
        
        if consolidated_report.empty:
            consolidated_report = report_renamed
        else:
            consolidated_report = pd.concat([consolidated_report, report_renamed], axis=1)
    output_file = "RAPPORT_GLOBAL_MULTI_TIMEFRAME.csv"
    consolidated_report.to_csv(output_file, sep=';')
    print(f"Saved: {output_file}")
    
  
if __name__ == "__main__":
    main()

>>> STARTING MULTI-TIMEFRAME ANALYSIS (1Y, 5Y, FULL) <<<
   -> Saved MATRICE_CORRELATION_1Y.csv
   -> Saved MATRICE_CORRELATION_5Y.csv
   -> Saved MATRICE_CORRELATION_FULL.csv
Saved: RAPPORT_GLOBAL_MULTI_TIMEFRAME.csv
